In [519]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd

In [520]:
df = gpd.read_file('surge10m_2023_04_48039_riskval4.shp')

In [521]:
df.shape

(335825, 10)

In [522]:
df.head()

,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,geometry
0,16844373,4,Very High,48039,0,1,1,1,1,"POLYGON ((-95.40796 28.98426, -95.40796 28.984..."
1,16844374,4,Very High,48039,0,1,1,1,1,"POLYGON ((-95.40843 28.98454, -95.40843 28.984..."
2,16844375,4,Very High,48039,0,1,1,1,1,"POLYGON ((-95.35204 28.97352, -95.35222 28.973..."
3,16844376,4,Very High,48039,0,1,1,1,1,"POLYGON ((-95.35185 28.97333, -95.35194 28.973..."
4,16844377,4,Very High,48039,0,1,1,1,1,"POLYGON ((-95.35176 28.97324, -95.35185 28.973..."


In [523]:
# Check topological relationships (e.g., adjacency)
df['neighbor'] = None

for i, row in df.iterrows():
    row_adj = []
    for j, row2 in df.iterrows():
        if row['geometry'].touches(row2['geometry']):
            row_adj.append(j)
    df.at[i, 'neighbor'] = row_adj

KeyboardInterrupt: 

In [ ]:
df.head()

,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,geometry,neighbor
0,17056189,4,Very High,48039,0,5,10,15,18,"POLYGON ((-95.43991 28.96676, -95.44009 28.966...","[3, 4, 37, 64]"
1,17056827,4,Very High,48039,0,5,10,15,18,"POLYGON ((-95.43741 28.96787, -95.43769 28.967...","[6, 29, 30, 47]"
2,17057446,4,Very High,48039,0,5,10,15,19,"POLYGON ((-95.43741 28.96759, -95.43741 28.967...","[15, 29, 36, 47, 59]"
3,17057591,4,Very High,48039,0,5,10,15,19,"POLYGON ((-95.44074 28.96704, -95.44083 28.967...","[0, 13, 37, 38, 61, 68]"
4,17057779,4,Very High,48039,0,5,10,15,19,"POLYGON ((-95.43963 28.96667, -95.43991 28.966...","[0, 7, 64]"


In [ ]:
df['total_cat_depths'] = df['CAT4DEP'] + df['CAT5DEP']

In [ ]:
df.head()

,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,geometry,neighbor,total_cat_depths
0,17056189,4,Very High,48039,0,5,10,15,18,"POLYGON ((-95.43991 28.96676, -95.44009 28.966...","[3, 4, 37, 64]",33
1,17056827,4,Very High,48039,0,5,10,15,18,"POLYGON ((-95.43741 28.96787, -95.43769 28.967...","[6, 29, 30, 47]",33
2,17057446,4,Very High,48039,0,5,10,15,19,"POLYGON ((-95.43741 28.96759, -95.43741 28.967...","[15, 29, 36, 47, 59]",34
3,17057591,4,Very High,48039,0,5,10,15,19,"POLYGON ((-95.44074 28.96704, -95.44083 28.967...","[0, 13, 37, 38, 61, 68]",34
4,17057779,4,Very High,48039,0,5,10,15,19,"POLYGON ((-95.43963 28.96667, -95.43991 28.966...","[0, 7, 64]",34


In [ ]:

# Define a similarity metric and threshold (e.g., attribute difference)
similarity_threshold = 1  # Adjust as needed

# Identify similar neighbors

for index, row in df.iterrows():
    similar_neighbors = []
    neighbors = df.loc[df.index.isin(row['neighbor'])]
    for j, neighbor in neighbors.iterrows():
        if abs(row['total_cat_depths'] - neighbor['total_cat_depths']) <= similarity_threshold:
            df.at[index, 'total_cat_depths_corrected'] = neighbor['total_cat_depths']
    #print(index)
    #print(similar_neighbors)
    #df.at[index, 'similar_neighbors'] = similar_neighbors

# Visualize the results if needed


In [ ]:
sets = df['total_cat_depths_corrected'].value_counts().index

In [ ]:
sets

Float64Index([36.0, 38.0, 37.0, 35.0, 34.0], dtype='float64')

In [ ]:
df = df.drop(['neighbor'], axis=1)
df_catdep = df.drop('geometry', axis=1)
df_geom = df['geometry']
df_catdep.head()

,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_depths,total_cat_depths_corrected
0,17056189,4,Very High,48039,0,5,10,15,18,33,34.0
1,17056827,4,Very High,48039,0,5,10,15,18,33,34.0
2,17057446,4,Very High,48039,0,5,10,15,19,34,35.0
3,17057591,4,Very High,48039,0,5,10,15,19,34,34.0
4,17057779,4,Very High,48039,0,5,10,15,19,34,34.0


In [ ]:
for s in sets:
    print('Set: ', s)
    set_df = df_catdep.loc[(df_catdep['total_cat_depths_corrected'] == s)]
    matching_rows = set_df[(set_df['total_cat_depths_corrected'] == s) & (set_df['total_cat_depths'] == s)]
    rows_to_change = set_df[set_df['total_cat_depths'] != s ]
    
    # Update those rows to match rows where total_cat_depths == 34
    for index, row in rows_to_change.iterrows():
        if not matching_rows.empty:
            for col in matching_rows.columns:
                val = matching_rows.iloc[0][col]
                #print('col: ', col, 'val: ', val)
                df_catdep.loc[index, col] = val

    


Set:  36.0
Set:  38.0
Set:  37.0
Set:  35.0
Set:  34.0


In [ ]:
df_catdep = df_catdep.drop(['total_cat_depths_corrected'], axis=1)
df_catdep.head()

,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_depths
0,17057591,4,Very High,48039,0,5,10,15,19,34
1,17057591,4,Very High,48039,0,5,10,15,19,34
2,17057446,4,Very High,48039,0,5,10,15,19,34
3,17057591,4,Very High,48039,0,5,10,15,19,34
4,17057779,4,Very High,48039,0,5,10,15,19,34


In [ ]:
print(df_geom.shape)
print(df_catdep.shape)

(89,)
(89, 10)


In [ ]:
# join the two dataframes
final_df = df_catdep.join(df_geom)
final_df.head()

,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP,total_cat_depths,geometry
0,17057591,4,Very High,48039,0,5,10,15,19,34,"POLYGON ((-95.43991 28.96676, -95.44009 28.966..."
1,17057591,4,Very High,48039,0,5,10,15,19,34,"POLYGON ((-95.43741 28.96787, -95.43769 28.967..."
2,17057446,4,Very High,48039,0,5,10,15,19,34,"POLYGON ((-95.43741 28.96759, -95.43741 28.967..."
3,17057591,4,Very High,48039,0,5,10,15,19,34,"POLYGON ((-95.44074 28.96704, -95.44083 28.967..."
4,17057779,4,Very High,48039,0,5,10,15,19,34,"POLYGON ((-95.43963 28.96667, -95.43991 28.966..."


In [ ]:
final_df = gpd.GeoDataFrame(final_df, geometry='geometry', crs=4326)

In [ ]:
final_dissolved = final_df.dissolve(by=['total_cat_depths'])

,geometry,ID,RISK_VALUE,SURGE_RISK,FIPSSTCO,CAT1DEP,CAT2DEP,CAT3DEP,CAT4DEP,CAT5DEP
total_cat_depths,,,,,,,,,,
34,"MULTIPOLYGON (((-95.44065 28.96657, -95.43991 ...",17057591,4,Very High,48039,0,5,10,15,19
36,"MULTIPOLYGON (((-95.44102 28.96704, -95.44102 ...",17110199,4,Very High,48039,0,6,11,16,20
38,"MULTIPOLYGON (((-95.43917 28.96694, -95.43917 ...",17147717,4,Very High,48039,0,7,12,17,21


In [ ]:
explode = final_dissolved.explode(index_parts=True)

ID  RISK_VALUE SURGE_RISK FIPSSTCO  CAT1DEP  \
total_cat_depths                                                         
34               0   17057591           4  Very High    48039        0   
                 1   17057591           4  Very High    48039        0   
                 2   17057591           4  Very High    48039        0   
                 3   17057591           4  Very High    48039        0   
                 4   17057591           4  Very High    48039        0   
                 5   17057591           4  Very High    48039        0   
                 6   17057591           4  Very High    48039        0   
                 7   17057591           4  Very High    48039        0   
                 8   17057591           4  Very High    48039        0   
                 9   17057591           4  Very High    48039        0   
                 10  17057591           4  Very High    48039        0   
36               0   17110199           4  Very High    48039        0   
                 1   17110199           4  Very High    48039        0   
                 2   17110199           4  Very High    48039        0   
                 3   17110199           4  Very High    48039        0   
                 4   17110199           4  Very High    48039        0   
                 5   17110199           4  Very High    48039        0   
                 6   17110199           4  Very High    48039        0   
                 7   17110199           4  Very High    48039        0   
                 8   17110199           4  Very High    48039        0   
                 9   17110199           4  Very High    48039        0   
                 10  17110199           4  Very High    48039        0   
                 11  17110199           4  Very High    48039        0   
                 12  17110199           4  Very High    48039        0   
                 13  17110199           4  Very High    48039        0   
38               0   17147717           4  Very High    48039        0   
                 1   17147717           4  Very High    48039        0   
                 2   17147717           4  Very High    48039        0   

                     CAT2DEP  CAT3DEP  CAT4DEP  CAT5DEP  \
total_cat_depths                                          
34               0         5       10       15       19   
                 1         5       10       15       19   
                 2         5       10       15       19   
                 3         5       10       15       19   
                 4         5       10       15       19   
                 5         5       10       15       19   
                 6         5       10       15       19   
                 7         5       10       15       19   
                 8         5       10       15       19   
                 9         5       10       15       19   
                 10        5       10       15       19   
36               0         6       11       16       20   
                 1         6       11       16       20   
                 2         6       11       16       20   
                 3         6       11       16       20   
                 4         6       11       16       20   
                 5         6       11       16       20   
                 6         6       11       16       20   
                 7         6       11       16       20   
                 8         6       11       16       20   
                 9         6       11       16       20   
                 10        6       11       16       20   
                 11        6       11       16       20   
                 12        6       11       16       20   
                 13        6       11       16       20   
38               0         7       12       17       21   
                 1         7       12       17       21   
                 2         7       12       17       21   

                                   

In [ ]:
explode = explode.reset_index()
explode = explode.drop(['total_cat_depths', 'level_1', 'ID'], axis=1)
explode['ID'] = explode.index

In [ ]:
explode.columns

Index(['RISK_VALUE', 'SURGE_RISK', 'FIPSSTCO', 'CAT1DEP', 'CAT2DEP', 'CAT3DEP',
       'CAT4DEP', 'CAT5DEP', 'geometry', 'ID'],
      dtype='object')

In [ ]:
explode[['ID', 'RISK_VALUE', 'SURGE_RISK', 'FIPSSTCO', 'CAT1DEP', 'CAT2DEP', 'CAT3DEP',
       'CAT4DEP', 'CAT5DEP', 'geometry']].to_file('ss_riskval4_dissolved.shp')